<a href="https://colab.research.google.com/github/ncourtel1/strava-urban-mobility-pipeline/blob/main/notebook/01_pipeline_strava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stravalib pandas polyline requests folium python-dotenv stravaio wearipedia

In [ ]:
import pandas as pd
from stravalib import Client
import datetime
import polyline
import urllib
import requests
import json
from stravaio import StravaIO
import folium
import logging
import wearipedia

## 2. Authentication and Authorization
To obtain access to data, authorization is required. Put in your client id, client secret token and refresh token for your Strava account. We'll use this to extract the data in the sections below.

In [62]:
from dotenv import load_dotenv
import os

# Charger le fichier .env
load_dotenv('.env')

# Récupérer les variables
client_id = os.getenv('STRAVA_CLIENT_ID')
client_secret = os.getenv('STRAVA_CLIENT_SECRET')
access_token = os.getenv('STRAVA_ACCESS_TOKEN')
refresh_token = os.getenv('STRAVA_REFRESH_TOKEN')

In [63]:
params_oauth = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": f"http://localhost:8000/authorization_successful",
    "scope": "read,profile:read_all,activity:read",
    "approval_prompt": "force"
}
values_url = urllib.parse.urlencode(params_oauth)
base_url = 'https://www.strava.com/oauth/authorize'
authorize_url = base_url + '?' + values_url
print(authorize_url)

https://www.strava.com/oauth/authorize?client_id=181205&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2Fauthorization_successful&scope=read%2Cprofile%3Aread_all%2Cactivity%3Aread&approval_prompt=force


In [64]:
AUTHORIZATION_CODE = "b309fa2b32c7d10f6e200989d283a3d9c5d6bf6d" #@param {type: "string"}

In [71]:
#@title Enter Strava API credentials

def get_strava_refresh_token(client_id, client_secret, code):
    """
    Get a refresh token from Strava API using authorization code.

    Args:
        client_id (str): Your Strava API client ID
        client_secret (str): Your Strava API client secret
        code (str): Authorization code received from Strava

    Returns:
        str: The refresh token
    """
    url = "https://www.strava.com/api/v3/oauth/token"
    payload = {
        'client_id': client_id,
        'client_secret': client_secret,
        'code': AUTHORIZATION_CODE,
        'grant_type': 'authorization_code'
    }

    try:
        response = requests.post(url, data=payload)
        response.raise_for_status()
        data = response.json()

        refresh_token = data.get('refresh_token')

        if not refresh_token:
            print("Warning: No refresh token found in response")
        return refresh_token

    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

# If you have a refresh token already directly, you can fill this in instead of calling the above function.
refresh_token = get_strava_refresh_token(client_id, client_secret, AUTHORIZATION_CODE)
print("Your refresh_token is", refresh_token)

# Note that the authorization code can only be used once, so you should hold onto this refresh token, or you can go through the authorize page

Error making request: 400 Client Error: Bad Request for url: https://www.strava.com/api/v3/oauth/token
Your refresh_token is None


In [69]:
client = StravaIO(access_token=access_token)
athlete = client.get_logged_in_athlete()
activities = client.get_logged_in_athlete_activities()

2025-10-16 15:15:51.092 | ERROR    | stravaio:get_logged_in_athlete:41 - "
            Error in strava_swagger_client.AthletesApi! 
            STRAVA_ACCESS_TOKEN is likely out of date!
            Check the https://github.com/sladkovm/strava-oauth for help.
            Returning None.
            Original Error:
            (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Thu, 16 Oct 2025 15:15:51 GMT', 'x-envoy-upstream-service-time': '14', 'server': 'istio-envoy', 'status': '401 Unauthorized', 'cache-control': 'no-cache', 'vary': 'Accept, Origin', 'referrer-policy': 'strict-origin-when-cross-origin', 'x-permitted-cross-domain-policies': 'none', 'x-xss-protection': '1; mode=block', 'x-request-id': '87e7a2a2-ee80-4e42-9993-51a7b261b5f8', 'x-download-options': 'noopen', 'x-frame-options': 'DENY', 'x-content-type-options': 'nosniff', 'X-Cache': 'Error

ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Thu, 16 Oct 2025 15:15:51 GMT', 'x-envoy-upstream-service-time': '45', 'server': 'istio-envoy', 'status': '401 Unauthorized', 'cache-control': 'no-cache', 'vary': 'Accept, Origin', 'referrer-policy': 'strict-origin-when-cross-origin', 'x-permitted-cross-domain-policies': 'none', 'x-xss-protection': '1; mode=block', 'x-request-id': '5a4b6850-fd35-4ee7-9975-d1b6e99f36e3', 'x-download-options': 'noopen', 'x-frame-options': 'DENY', 'x-content-type-options': 'nosniff', 'X-Cache': 'Error from cloudfront', 'Via': '1.1 ad1a856488fdab864472ba6d5677e39c.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'SEA900-P6', 'X-Amz-Cf-Id': 'QdIji2ZQMJzO5rb3bHnIdn8F2d-pVk0JV1OLnZYksT5hXWdHehXORg=='})
HTTP response body: {"message":"Authorization Error","errors":[{"resource":"Athlete","field":"access_token","code":

## 3. Data Extraction

In [ ]:
#set start and end dates
start_date='2021-09-01' #@param {type:"string"}
end_date='2025-10-31' #@param {type:"string"}
synthetic = False #@param {type:"boolean"}

In [ ]:
device = wearipedia.get_device("strava/strava")

if not synthetic:
    device.authenticate({
    'client_id':client_id,
    'client_secret':client_secret,
    'refresh_token':refresh_token
    })

params = {"start_date": start_date, "end_date": end_date}

start_latlng = device.get_data("start_latlng", params=params)
end_latlng = device.get_data("end_latlng", params=params)
distance = device.get_data("distance", params=params)
moving_time = device.get_data("moving_time", params=params)
elapsed_time = device.get_data("elapsed_time", params=params)
total_elevation_gain = device.get_data("total_elevation_gain", params=params)
average_speed = device.get_data("average_speed", params=params)
max_speed = device.get_data("max_speed", params=params)
average_heartrate = device.get_data("average_heartrate", params=params)
max_heartrate = device.get_data("max_heartrate", params=params)
map_summary_polyline = device.get_data("map_summary_polyline", params=params)
elev_high = device.get_data("elev_high", params=params)
elev_low = device.get_data("elev_low", params=params)
average_cadence = device.get_data("average_cadence", params=params)
average_watts = device.get_data("average_watts", params=params)
kilojoules = device.get_data("kilojoules", params=params)


  return datetime.utcnow().replace(tzinfo=utc)



Requesting Token...

Access Token = 8dd63b88fd47b1cadf92f824cfaac5191a95fda9



ValueError: data_type must be in ['distance', 'moving_time', 'elapsed_time', 'total_elevation_gain', 'average_speed', 'max_speed', 'average_heartrate', 'max_heartrate', 'map_summary_polyline', 'elev_high', 'elev_low', 'average_cadence', 'average_watts', 'kilojoules', 'heartrate']

## 4. Data Exporting

Exporting to CSV/XLSX

We will thus export steps, heart rates, and breath rates all as separate files.

In [ ]:
complete = {
    "distance": distance,
    'moving_time':moving_time,
    'elapsed_time':elapsed_time,
    'total_elevation_gain':total_elevation_gain,
    'average_speed':average_speed,
    'max_speed':max_speed,
    'average_heartrate':average_heartrate,
    'max_heartrate':max_heartrate,
    'map_summary_polyline':map_summary_polyline,
    'elev_high':elev_high,
    'elev_low':elev_low,
    'average_cadence':average_cadence,
    'average_watts':average_watts,
    'kilojoules':kilojoules
}

In [ ]:
distance_df = pd.DataFrame.from_dict(complete['distance'])
distance_df.to_csv('distance.csv')
distance_df.to_excel('distance.xlsx')

moving_time_df = pd.DataFrame.from_dict(complete['moving_time'])
moving_time_df.to_csv('moving_time.csv')
moving_time_df.to_excel('moving_time.xlsx')

elapsed_time_df = pd.DataFrame.from_dict(complete['elapsed_time'])
elapsed_time_df.to_csv('elapsed_time.csv')
elapsed_time_df.to_excel('elapsed_time.xlsx')

total_elevation_gain_df = pd.DataFrame.from_dict(complete['total_elevation_gain'])
total_elevation_gain_df.to_csv('total_elevation_gain.csv')
total_elevation_gain_df.to_excel('total_elevation_gain.xlsx')

average_speed_df = pd.DataFrame.from_dict(complete['average_speed'])
average_speed_df.to_csv('average_speed.csv')
average_speed_df.to_excel('average_speed.xlsx')

max_speed_df = pd.DataFrame.from_dict(complete['max_speed'])
max_speed_df.to_csv('max_speed.csv')
max_speed_df.to_excel('max_speed.xlsx')

average_heartrate_df = pd.DataFrame.from_dict(complete['average_heartrate'])
average_heartrate_df.to_csv('average_heartrate.csv')
average_heartrate_df.to_excel('average_heartrate.xlsx')

max_heartrate_df = pd.DataFrame.from_dict(complete['max_heartrate'])
max_heartrate_df.to_csv('max_heartrate.csv')
max_heartrate_df.to_excel('max_heartrate.xlsx')

map_summary_polyline_df = pd.DataFrame.from_dict(complete['map_summary_polyline'])
map_summary_polyline_df.to_csv('map_summary_polyline.csv')
map_summary_polyline_df.to_excel('map_summary_polyline.xlsx')

elev_high_df = pd.DataFrame.from_dict(complete['elev_high'])
elev_high_df.to_csv('elev_high.csv')
elev_high_df.to_excel('elev_high.xlsx')

elev_low_df = pd.DataFrame.from_dict(complete['elev_low'])
elev_low_df.to_csv('elev_low.csv')
elev_low_df.to_excel('elev_low.xlsx')

average_cadence_df = pd.DataFrame.from_dict(complete['average_cadence'])
average_cadence_df.to_csv('average_cadence.csv')
average_cadence_df.to_excel('average_cadence.xlsx')

average_watts_df = pd.DataFrame.from_dict(complete['average_watts'])
average_watts_df.to_csv('average_watts.csv')
average_watts_df.to_excel('average_watts.xlsx')

kilojoules_df = pd.DataFrame.from_dict(complete['kilojoules'])
kilojoules_df.to_csv('kilojoules.csv')
kilojoules_df.to_excel('kilojoules.xlsx')


## Visualization

In [ ]:
m = folium.Map(
    tiles="cartodbpositron",
    location=[49.4435, 1.0984],
    zoom_start=11.5,
    control_scale=True
)
folium.TileLayer("cartodbpositron").add_to(m)
folium.TileLayer("cartodbdark_matter").add_to(m)
folium.LayerControl().add_to(m)

In [ ]:
def downsample(l, n):
    """Returns every nth element from list l. Returns the
    original list if n is set to 1.
    Used to reduce the number of GPS points per activity,
    to improve performance of the website.
    """

    return l[0::n]

def map_activities(activities, folium_map, opacity, weight):
    if len(activities) == 0:
        logging.info("No activities found, returning empty folium map.")
        return folium_map

    for a in activities:
        if a.type == "Workout":
            continue
        streams = client.get_activity_streams(a.id, athlete.id)
        try:
            points = list(zip(streams.lat, streams.lng))
            points = downsample(l=points, n=2)
            if a.type == "Run":
                folium.PolyLine(
                    locations=points, color="#ff9933", opacity=opacity, weight=weight
                ).add_to(folium_map)
            elif a.type == "Ride":
                folium.PolyLine(
                    locations=points, color="#0066ff", opacity=opacity, weight=weight
                ).add_to(folium_map)
            else:
                folium.PolyLine(
                    locations=points, color="#cc00ff", opacity=opacity, weight=weight
                ).add_to(folium_map)
            logging.critical("Mapped activity with id: {}".format(a.id))
        except Exception:
            logging.error("Could not map activity with id: {}".format(a.id))

    return folium_map

In [ ]:
m = map_activities(
    activities=activities,
    folium_map=m,
    opacity=0.5,
    weight=1
)

In [ ]:
m